# Гипотезы и их проверка

## Вспомогательные функции
### Импорты

In [ ]:
import kagglehub
import os
import cv2
import numpy as np
from typing import List
from src.features import (
    HorizontalEdgeCount,
    VerticalEdgeCount,
    CornerCount,
    GreenPixelPercentage,
    BluePixelPercentage,
    ContrastMeasure,
    TextureComplexity,
    SkyPixelRatio,
    ShadowPresence,
    SymmetryMeasure,
    SharpnessMeasure,
    AverageBrightness,
)
from scipy.stats import (
    ttest_ind,
    ks_2samp,
    mannwhitneyu,
)

### Загрузка датасета

In [ ]:
# Download latest version
path = os.path.join(
    kagglehub.dataset_download("rahmasleam/intel-image-dataset"), "Intel Image Dataset"
)

print("Путь к файлам датасета:", path)

### Функция загрузки изображений из директории

In [ ]:
def load_images_from_folder(folder: str) -> List[np.ndarray]:
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
    return images

### Объявление категорий

In [ ]:
categories = ["forest", "glacier", "street", "sea"]

### Функция загрузки изображений всех категорий, кроме указанной

In [ ]:
def load_other_categories_images(exclude_category: str) -> List[np.ndarray]:
    images = []

    for category in categories:
        if category != exclude_category:
            category_path = os.path.join(path, category)
            images.extend(load_images_from_folder(category_path))

    return images

### Функция вывода результатов проверки

In [ ]:
# Significance level for p-value
ALPHA = 0.05


def print_test_results(
    test_results: List[tuple[str, tuple]], success_message: str, failure_message: str
) -> None:
    for test_name, stat, p_value in test_results:
        print(f"{test_name}:")
        print(f"Statistic: {stat}")
        print(f"P-value: {p_value}\n")

    if all([p_value < ALPHA for _, _, p_value in test_results]):
        print(f"Гипотеза подтверждена: {success_message}")
    else:
        print(f"Гипотеза не подтверждена: {failure_message}")

## Проверка гипотез

### Гипотеза 1: Улицы имеют больше горизонтальных линий по сравнению с другими категориями.

В данной гипотезе предполагается, что изображения улиц содержат больше горизонтальных краев (например, дороги, бордюры и здания), чем изображения других категорий, таких как леса, ледники или моря.

### Способы проверки:

1. **T-тест**:
   Сравнение средних значений количества горизонтальных краев между категориями "street" и другими категориями. Если p-value < 0.05, то это будет свидетельствовать о статистически значимой разнице.

2. **KS тест (Колмогорова-Смирнова)**:
   Сравнение распределений количества горизонтальных краев между категориями. Тест проверяет, различается ли форма распределения данных.

3. **Тест Манна-Уитни U**:
   Анализ медианных значений количества горизонтальных краев для двух групп, чтобы определить различия, если данные имеют ненормальное распределение.

In [ ]:
# Instantiate the feature extractor
feature_extractor = HorizontalEdgeCount()

category = "street"
street_path = os.path.join(path, category)

# Load images from directories
street_images = load_images_from_folder(street_path)
other_images = load_other_categories_images(exclude_category=category)

print(len(street_images), len(other_images))

# Calculate features for both sets of images
street_features = [feature_extractor.calculate(img) for img in street_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

t_stat, p_value_ttest = ttest_ind(street_features, other_features, equal_var=False)
ks_stat, p_value_ks = ks_2samp(street_features, other_features)
u_stat, p_value_u = mannwhitneyu(
    street_features, other_features, alternative="two-sided"
)

# Collect test results
test_results = [
    ("T-тест", t_stat, p_value_ttest),
    ("KS тест", ks_stat, p_value_ks),
    ("Критерий Манна-Уитни U", u_stat, p_value_u),
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Улицы имеют больше горизонтальных линий.",
    failure_message="Значимой разницы не обнаружено.",
)

### Гипотеза 2: Ледники имеют больше вертикальных линий по сравнению с другими категориями.

Предполагается, что изображения ледников будут содержать больше вертикальных краев из-за текстур льда, трещин и вертикальных элементов пейзажа, чем изображения других категорий.

### Способы проверки:

1. **T-тест**:
   Сравнение средних значений количества вертикальных краев между категориями "glacier" и другими категориями. Если p-value < 0.05, то гипотеза подтверждается.

2. **KS тест (Колмогорова-Смирнова)**:
   Сравнение распределений количества вертикальных краев для оценки различий в формах распределений данных.

3. **Тест Манна-Уитни U**:
   Проверка медианных значений количества вертикальных краев для категорий, где данные могут иметь ненормальное распределение.

In [ ]:
# Instantiate the feature extractor
feature_extractor = VerticalEdgeCount()

category = "glacier"
glacier_path = os.path.join(path, category)

# Load images from directories
glacier_images = load_images_from_folder(glacier_path)
other_images = load_other_categories_images(exclude_category=category)

print(len(glacier_images), len(other_images))

# Calculate features for both sets of images
glacier_features = [feature_extractor.calculate(img) for img in glacier_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

t_stat, p_value_ttest = ttest_ind(glacier_features, other_features, equal_var=False)
ks_stat, p_value_ks = ks_2samp(glacier_features, other_features)
u_stat, p_value_u = mannwhitneyu(
    glacier_features, other_features, alternative="two-sided"
)

# Collect test results
test_results = [
    ("T-тест", t_stat, p_value_ttest),
    ("KS тест", ks_stat, p_value_ks),
    ("Критерий Манна-Уитни U", u_stat, p_value_u),
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Ледники имеют больше вертикальных линий.",
    failure_message="Значимой разницы не обнаружено.",
)

### Гипотеза 3: Изображения улиц содержат больше углов (corner points), чем другие категории.

Предполагается, что изображения улиц будут содержать больше угловых точек из-за сложной структуры зданий, дорожной разметки и других элементов городской среды.

### Способы проверки:

1. **T-тест**:
   Сравнение средних значений количества угловых точек между изображениями улиц и других категорий. p-value < 0.05 подтвердит гипотезу.

2. **KS тест (Колмогорова-Смирнова)**:
   Анализ различий в распределении количества угловых точек между категориями.

3. **Тест Манна-Уитни U**:
   Проверка различий в медианных значениях между категориями, чтобы оценить статистическую значимость различий.

In [ ]:
# Instantiate the feature extractor
feature_extractor = CornerCount()

category = "street"
street_path = os.path.join(path, category)

# Load images from directories
street_images = load_images_from_folder(street_path)
other_images = load_other_categories_images(exclude_category=category)

print(len(street_images), len(other_images))

# Calculate features for both sets of images
street_features = [feature_extractor.calculate(img) for img in street_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

t_stat, p_value_ttest = ttest_ind(street_features, other_features, equal_var=False)
ks_stat, p_value_ks = ks_2samp(street_features, other_features)
u_stat, p_value_u = mannwhitneyu(
    street_features, other_features, alternative="two-sided"
)

# Collect test results
test_results = [
    ("T-тест", t_stat, p_value_ttest),
    ("KS тест", ks_stat, p_value_ks),
    ("Критерий Манна-Уитни U", u_stat, p_value_u),
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Улицы содержат больше углов.",
    failure_message="Значимой разницы не обнаружено.",
)

### Гипотеза 4: Леса содержат больше зелёных пикселей по сравнению с другими категориями.

В данной гипотезе предполагается, что изображения лесов будут иметь значительно больше пикселей зелёного цвета, что связано с наличием деревьев, травы и других природных объектов в данной категории.

### Способы проверки:

1. **T-тест**:
   Сравнение средних значений доли зелёных пикселей между изображениями категории "forest" и других категорий. Если p-value < 0.05, то гипотеза подтверждается.

2. **KS тест (Колмогорова-Смирнова)**:
   Анализ различий в распределении доли зелёных пикселей между категориями.

3. **Тест Манна-Уитни U**:
   Проверка медианных значений доли зелёных пикселей между категориями, чтобы оценить статистическую значимость различий.

In [ ]:
# Instantiate the feature extractor
feature_extractor = GreenPixelPercentage()

category = "forest"
forest_path = os.path.join(path, category)

# Load images from directories
forest_images = load_images_from_folder(forest_path)
other_images = load_other_categories_images(exclude_category=category)

print(len(forest_images), len(other_images))

# Calculate features for both sets of images
forest_features = [feature_extractor.calculate(img) for img in forest_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

t_stat, p_value_ttest = ttest_ind(forest_features, other_features, equal_var=False)
ks_stat, p_value_ks = ks_2samp(forest_features, other_features)
u_stat, p_value_u = mannwhitneyu(
    forest_features, other_features, alternative="two-sided"
)

# Collect test results
test_results = [
    ("T-тест", t_stat, p_value_ttest),
    ("KS тест", ks_stat, p_value_ks),
    ("Критерий Манна-Уитни U", u_stat, p_value_u),
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Леса содержат больше зелёных пикселей.",
    failure_message="Значимой разницы не обнаружено.",
)

### Гипотеза 5: Моря содержат больше синих пикселей по сравнению с другими категориями.

Предполагается, что изображения морей будут иметь значительно больше синих пикселей из-за воды и отражения неба, которые являются ключевыми элементами морских пейзажей.

### Способы проверки:

1. **T-тест**:
   Сравнение средних значений доли синих пикселей между изображениями категории "sea" и других категорий. p-value < 0.05 подтвердит гипотезу.

2. **KS тест (Колмогорова-Смирнова)**:
   Сравнение распределений доли синих пикселей между категориями.

3. **Тест Манна-Уитни U**:
   Проверка различий в медианных значениях доли синих пикселей между категориями.

In [ ]:
# Instantiate the feature extractor
feature_extractor = BluePixelPercentage()

category = "sea"
sea_path = os.path.join(path, category)

# Load images from directories
sea_images = load_images_from_folder(sea_path)
other_images = load_other_categories_images(exclude_category=category)

print(len(sea_images), len(other_images))

# Calculate features for both sets of images
sea_features = [feature_extractor.calculate(img) for img in sea_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

t_stat, p_value_ttest = ttest_ind(sea_features, other_features, equal_var=False)
ks_stat, p_value_ks = ks_2samp(sea_features, other_features)
u_stat, p_value_u = mannwhitneyu(sea_features, other_features, alternative="two-sided")

# Collect test results
test_results = [
    ("T-тест", t_stat, p_value_ttest),
    ("KS тест", ks_stat, p_value_ks),
    ("Критерий Манна-Уитни U", u_stat, p_value_u),
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Моря содержат больше синих пикселей.",
    failure_message="Значимой разницы не обнаружено.",
)

### Гипотеза 6: Ледники имеют более высокий контраст по сравнению с другими категориями.

Предполагается, что изображения ледников будут иметь более высокий уровень контраста из-за ярких белых областей снега и тёмных теней, создаваемых текстурой льда и окружающим ландшафтом.

### Способы проверки:

1. **T-тест**:
   Сравнение средних значений уровня контраста между изображениями категории "glacier" и других категорий. p-value < 0.05 подтверждает гипотезу.

2. **KS тест (Колмогорова-Смирнова)**:
   Анализ различий в распределении уровня контраста между категориями.

3. **Тест Манна-Уитни U**:
   Проверка медианных значений уровня контраста для категорий, чтобы выявить статистически значимые различия.

In [ ]:
# Instantiate the feature extractor
feature_extractor = ContrastMeasure()

category = "glacier"
glacier_path = os.path.join(path, category)

# Load images from directories
glacier_images = load_images_from_folder(glacier_path)
other_images = load_other_categories_images(exclude_category=category)

print(len(glacier_images), len(other_images))

# Calculate features for both sets of images
glacier_features = [feature_extractor.calculate(img) for img in glacier_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

t_stat, p_value_ttest = ttest_ind(glacier_features, other_features, equal_var=False)
ks_stat, p_value_ks = ks_2samp(glacier_features, other_features)
u_stat, p_value_u = mannwhitneyu(
    glacier_features, other_features, alternative="two-sided"
)

# Collect test results
test_results = [
    ("T-тест", t_stat, p_value_ttest),
    ("KS тест", ks_stat, p_value_ks),
    ("Критерий Манна-Уитни U", u_stat, p_value_u),
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Ледники имеют более высокий контраст.",
    failure_message="Значимой разницы не обнаружено.",
)

### Гипотеза 7: Изображения улиц имеют более сложную текстуру по сравнению с другими категориями.

Предполагается, что изображения улиц характеризуются более высокой сложностью текстуры из-за присутствия множества различных объектов, таких как здания, дороги и автомобили, создающих разнообразные текстурные элементы.

### Способы проверки:

1. **T-тест**:
   Сравнение средних значений сложности текстуры между изображениями категории "street" и других категорий. Если p-value < 0.05, гипотеза подтверждается.

2. **KS тест (Колмогорова-Смирнова)**:
   Проверка различий в распределении значений сложности текстуры между категориями.

3. **Тест Манна-Уитни U**:
   Анализ различий в медианных значениях сложности текстуры между категориями.

In [ ]:
# Instantiate the feature extractor
feature_extractor = TextureComplexity()

category = "street"
street_path = os.path.join(path, category)

# Load images from directories
street_images = load_images_from_folder(street_path)
other_images = load_other_categories_images(exclude_category=category)

print(len(street_images), len(other_images))

# Calculate features for both sets of images
street_features = [feature_extractor.calculate(img) for img in street_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

t_stat, p_value_ttest = ttest_ind(street_features, other_features, equal_var=False)
ks_stat, p_value_ks = ks_2samp(street_features, other_features)
u_stat, p_value_u = mannwhitneyu(
    street_features, other_features, alternative="two-sided"
)

# Collect test results
test_results = [
    ("T-тест", t_stat, p_value_ttest),
    ("KS тест", ks_stat, p_value_ks),
    ("Критерий Манна-Уитни U", u_stat, p_value_u),
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Улицы имеют более сложную текстуру.",
    failure_message="Значимой разницы не обнаружено.",
)

### Гипотеза 8: Моря содержат большее соотношение пикселей, относящихся к небу, по сравнению с другими категориями.

Предполагается, что изображения категории "sea" содержат больше пикселей, относящихся к небу, из-за открытого горизонта и отражения неба в воде.

### Способы проверки:

1. **T-тест**:
   Сравнение средних значений отношения пикселей неба между изображениями категории "sea" и других категорий. p-value < 0.05 подтверждает гипотезу.

2. **KS тест (Колмогорова-Смирнова)**:
   Проверка распределений соотношения пикселей неба между категориями.

3. **Тест Манна-Уитни U**:
   Анализ различий медианных значений соотношения пикселей неба между категориями.


In [ ]:
# Instantiate the feature extractor
feature_extractor = SkyPixelRatio()

category = "sea"
sea_path = os.path.join(path, category)

# Load images from directories
sea_images = load_images_from_folder(sea_path)
other_images = load_other_categories_images(exclude_category=category)

print(len(sea_images), len(other_images))

# Calculate features for both sets of images
sea_features = [feature_extractor.calculate(img) for img in sea_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

t_stat, p_value_ttest = ttest_ind(sea_features, other_features, equal_var=False)
ks_stat, p_value_ks = ks_2samp(sea_features, other_features)
u_stat, p_value_u = mannwhitneyu(sea_features, other_features, alternative="two-sided")

# Collect test results
test_results = [
    ("T-тест", t_stat, p_value_ttest),
    ("KS тест", ks_stat, p_value_ks),
    ("Критерий Манна-Уитни U", u_stat, p_value_u),
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Моря содержат большее соотношение пикселей неба.",
    failure_message="Значимой разницы не обнаружено.",
)

### Гипотеза 9: Ледники чаще содержат тени по сравнению с другими категориями.

Предполагается, что изображения категории "glacier" имеют большее присутствие теней, что связано с рельефной текстурой льда и снегом, создающим контрастные области.

### Способы проверки:

1. **T-тест**:
   Сравнение средних значений присутствия теней между изображениями категории "glacier" и других категорий. Если p-value < 0.05, гипотеза подтверждается.

2. **KS тест (Колмогорова-Смирнова)**:
   Проверка распределений значений присутствия теней между категориями.

3. **Тест Манна-Уитни U**:
   Анализ различий медианных значений присутствия теней между категориями.

In [ ]:
# Instantiate the feature extractor
feature_extractor = ShadowPresence()

category = "glacier"
glacier_path = os.path.join(path, category)

# Load images from directories
glacier_images = load_images_from_folder(glacier_path)
other_images = load_other_categories_images(exclude_category=category)

print(len(glacier_images), len(other_images))

# Calculate features for both sets of images
glacier_features = [feature_extractor.calculate(img) for img in glacier_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

t_stat, p_value_ttest = ttest_ind(glacier_features, other_features, equal_var=False)
ks_stat, p_value_ks = ks_2samp(glacier_features, other_features)
u_stat, p_value_u = mannwhitneyu(
    glacier_features, other_features, alternative="two-sided"
)

# Collect test results
test_results = [
    ("T-тест", t_stat, p_value_ttest),
    ("KS тест", ks_stat, p_value_ks),
    ("Критерий Манна-Уитни U", u_stat, p_value_u),
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Ледники чаще содержат тени.",
    failure_message="Значимой разницы не обнаружено.",
)

### Гипотеза 10: Леса характеризуются более высокой симметрией по сравнению с другими категориями.

Предполагается, что изображения категории "forest" имеют более высокую симметрию из-за структур деревьев и общего распределения объектов в природе.

### Способы проверки:

1. **T-тест**:
   Сравнение средних значений меры симметрии между изображениями категории "forest" и других категорий. Если p-value < 0.05, гипотеза подтверждается.

2. **KS тест (Колмогорова-Смирнова)**:
   Проверка различий в распределении меры симметрии между категориями.

3. **Тест Манна-Уитни U**:
   Анализ различий медианных значений меры симметрии между категориями.


In [ ]:
# Instantiate the feature extractor
feature_extractor = SymmetryMeasure()

category = "forest"
forest_path = os.path.join(path, category)

# Load images from directories
forest_images = load_images_from_folder(forest_path)
other_images = load_other_categories_images(exclude_category=category)

print(len(forest_images), len(other_images))

# Calculate features for both sets of images
forest_features = [feature_extractor.calculate(img) for img in forest_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

t_stat, p_value_ttest = ttest_ind(forest_features, other_features, equal_var=False)
ks_stat, p_value_ks = ks_2samp(forest_features, other_features)
u_stat, p_value_u = mannwhitneyu(
    forest_features, other_features, alternative="two-sided"
)

# Collect test results
test_results = [
    ("T-тест", t_stat, p_value_ttest),
    ("KS тест", ks_stat, p_value_ks),
    ("Критерий Манна-Уитни U", u_stat, p_value_u),
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Леса характеризуются более высокой симметрией.",
    failure_message="Значимой разницы не обнаружено.",
)

### Гипотеза 11: Улицы имеют более высокую резкость по сравнению с другими категориями.

Предполагается, что изображения категории "street" характеризуются более высокой резкостью из-за наличия деталей, таких как здания, дороги и другие элементы городской среды.

### Способы проверки:

1. **T-тест**:
   Сравнение средних значений меры резкости между изображениями категории "street" и других категорий. Если p-value < 0.05, гипотеза подтверждается.

2. **KS тест (Колмогорова-Смирнова)**:
   Проверка различий в распределении меры резкости между категориями.

3. **Тест Манна-Уитни U**:
   Анализ различий медианных значений меры резкости между категориями.

In [ ]:
# Instantiate the feature extractor
feature_extractor = SharpnessMeasure()

category = "street"
street_path = os.path.join(path, category)

# Load images from directories
street_images = load_images_from_folder(street_path)
other_images = load_other_categories_images(exclude_category=category)

print(len(street_images), len(other_images))

# Calculate features for both sets of images
street_features = [feature_extractor.calculate(img) for img in street_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

t_stat, p_value_ttest = ttest_ind(street_features, other_features, equal_var=False)
ks_stat, p_value_ks = ks_2samp(street_features, other_features)
u_stat, p_value_u = mannwhitneyu(
    street_features, other_features, alternative="two-sided"
)

# Collect test results
test_results = [
    ("T-тест", t_stat, p_value_ttest),
    ("KS тест", ks_stat, p_value_ks),
    ("Критерий Манна-Уитни U", u_stat, p_value_u),
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Улицы имеют более высокую резкость.",
    failure_message="Значимой разницы не обнаружено.",
)

### Гипотеза 12: Моря имеют более высокую среднюю яркость по сравнению с другими категориями.

Предполагается, что изображения категории "sea" характеризуются более высокой средней яркостью из-за отражения солнечного света от воды и ясного неба.

### Способы проверки:

1. **T-тест**:
   Сравнение средних значений средней яркости между изображениями категории "sea" и других категорий. Если p-value < 0.05, гипотеза подтверждается.

2. **KS тест (Колмогорова-Смирнова)**:
   Проверка различий в распределении средней яркости между категориями.

3. **Тест Манна-Уитни U**:
   Анализ различий медианных значений средней яркости между категориями.

In [ ]:
# Instantiate the feature extractor
feature_extractor = AverageBrightness()

category = "sea"
sea_path = os.path.join(path, category)

# Load images from directories
sea_images = load_images_from_folder(sea_path)
other_images = load_other_categories_images(exclude_category=category)

print(len(sea_images), len(other_images))

# Calculate features for both sets of images
sea_features = [feature_extractor.calculate(img) for img in sea_images]
other_features = [feature_extractor.calculate(img) for img in other_images]

t_stat, p_value_ttest = ttest_ind(sea_features, other_features, equal_var=False)
ks_stat, p_value_ks = ks_2samp(sea_features, other_features)
u_stat, p_value_u = mannwhitneyu(sea_features, other_features, alternative="two-sided")

# Collect test results
test_results = [
    ("T-тест", t_stat, p_value_ttest),
    ("KS тест", ks_stat, p_value_ks),
    ("Критерий Манна-Уитни U", u_stat, p_value_u),
]

# Print results with conclusion
print_test_results(
    test_results,
    success_message="Моря имеют более высокую среднюю яркость.",
    failure_message="Значимой разницы не обнаружено.",
)

## Выводы

### Подтвержденные гипотезы:
- **Гипотеза 1**: Изображения улицы имеют больше горизонтальных линий.
- **Гипотеза 2**: Изображения ледников имеют больше вертикальных линий.
- **Гипотеза 4**: Изображения леса содержат больше зелёных пикселей.
- **Гипотеза 6**: Изображения ледников имеют более высокий контраст.
- **Гипотеза 9**: Ледники чаще содержат тени.
- **Гипотеза 10**: Леса характеризуются более высокой симметрией.
- **Гипотеза 11**: Улицы имеют более высокую резкость.
- **Гипотеза 12**: Моря имеют более высокую среднюю яркость.

### Опровергнутые гипотезы:
- **Гипотеза 3**: Изображения улиц содержат больше углов (corner points), чем другие категории. (Значимой разницы не обнаружено.)
- **Гипотеза 5**: Изображения моря содержат больше синих пикселей по сравнению с другими категориями. (Значимой разницы не обнаружено.)
- **Гипотеза 7**: Изображения улиц имеют более сложную текстуру по сравнению с другими категориями. (Значимой разницы не обнаружено.)
- **Гипотеза 8**: Изображения моря содержат большее соотношение пикселей, относящихся к небу, по сравнению с другими категориями. (Значимой разницы не обнаружено.)